# Code to plot the TTV diagram (Residual from the linear fit vs epoch)

Importing all the necessary packages. In this project, we basically used glob, pandas and bokeh. However, we didn't import the whole package. Instead we imported only the function that we need.

In [1]:
from glob import glob
from pandas import read_fwf
from bokeh.plotting import save, figure, output_file
from bokeh.models import ColumnDataSource, OpenURL, TapTool, BasicTickFormatter
from bokeh.models.tools import HoverTool
from bokeh.transform import factor_cmap
from bokeh.io import export_png

First, we fetch all the data files that is in "modified_data" folder. In the modified data folder, we added one more column to the original data file. This column indictes a light curve name for each 'ThisStudy' and just a random garbage value for others.

In [2]:
file_list = glob('modified_data/*.txt')

Then for each TTV data file, we plotted the linear residual (min) vs epoch (cycle). Here, we also added a hover effect and link to the light curve for 'ThisStudy' LCType. Follow the code comment for details.

In [3]:
for data_file in file_list:        
    
    output = "plots/ttv/" + data_file[14:-4] + ".html"                # creating an output html file
    output_file(output)

    df = read_fwf(data_file, skiprows=1, header=None, sep="\t+")      # importing the data file as a dataframe
    df = df.drop(df[df[21] == 1].index)                              # discarding all the rows with Discarded == 1
    
    #df = ref.drop(ref[ref[8] == 'ThisStudy'].index)         # Creating a dataframe for Database and Literature light curves
    This = df.drop(df[df[8] != 'ThisStudy'].index)           # Creating a dataframe for ThisStudy light curves

    source = ColumnDataSource(data=dict(                  # The residual is multiplied by 1440 to convert it from days to
                epoch = df[4],                            # minutes. err_l and err_h is also calculated to use later to plot 
                res = df[6]*1440,                         # the lower and higher value of the error bar segment. 
                LCType = df[8], 
                err_l = (df[6] - df[1])*1440,
                err_h = (df[6] + df[1])*1440
            ))

    sourceThis = ColumnDataSource(data=dict(           # This is same as above but for the ThisStudy LCType. We do it
                epoch=This[4],                         # seperately because ThisStudy data points have link, img_link and 
                res=This[6]*1440,                      # other properties that we need to show hover effect
                LCType=This[8],
        
                LCFileName=This[22], 
                img_link='../images/' + This[22] + '.png',
                link = '../lc/' + This[22] + '.html'
            ))

    p = figure(sizing_mode="stretch_both",                              # initiating a figure with titles, labels and tools
                tools="pan, box_zoom, save, reset, wheel_zoom, help",   # add/remove tools to show/hide in the bokeh plot
                title="TTV Diagram for " + data_file[14:-4],
                x_axis_label = "Epoch (cycle)", 
                y_axis_label = "O-C (min)")

    p.segment(                                  # Error bar plotting.
            x0='epoch',
            y0='err_l',
            x1='epoch',
            y1='err_h',
            color = 'black',
            source=source
        )
    
    p.circle('epoch', 'res',                                          # plotting Database and Literature data
                size=10,                                              # For new data such as TESS, Kepler etc, just add 
                source=source,                                        # more entry in the factor_cmap as 
                color=factor_cmap('LCType',
                                ['red', 'gray', 'turquoise', 'darkblue'],
                                ['ThisStudy', 'Database', 'Literature', 'TESS']),
                legend_group='LCType'
            )

    p.circle('epoch', 'res',                       # This is for ThisStudy light curves. The size is a little larger
                size=12,                           # and  red in color. Also, hover effect is enabled here.
                source=sourceThis,
                color=factor_cmap('LCType',
                                ['red'],
                                ['ThisStudy']),
                name='needsHover'
            )

    p.legend.title = "Light Curve Type"
    p.legend.location = "top_left"               # Choose the most suitable position for legend location

    hover = HoverTool(names=['needsHover'])      # This is the html code that will make the hover effect window
    hover.tooltips = """
          <div>
            <h3>@LCFileName</h3>
            <div><strong>Click to view this light curve</strong></div>
            <div><img src="@img_link" alt="Sorry, the image didn't load properly." width="200" /></div>
          </div>
        """
    p.add_tools(hover)
    
    url = "@link"                               # Here we add clicking effect so clicking each data point will open
    tap = TapTool(names=['needsHover'])         # a new tab with corresponding light curve
    tap.callback = OpenURL(url=url)
    p.add_tools(tap)
    
    p.xaxis.axis_label_text_font_size = "15pt"   # Setting up the font sizes
    p.yaxis.axis_label_text_font_size = "15pt"
    p.xaxis.major_label_text_font_size = "15pt"
    p.yaxis.major_label_text_font_size = "15pt"
    p.xaxis.formatter = BasicTickFormatter(use_scientific=False)
    p.yaxis.formatter = BasicTickFormatter(use_scientific=False)
    p.title.text_font_size = '20pt'
    p.legend.label_text_font_size = '13pt'
    p.legend.title_text_font_size = '14pt'
    
    export_png(p, filename="plots/ttv_images/" + data_file[14:-4] + ".png") # exporting as png image to use in the homepage
    save(p)